## ToC
- [smolLM](#smolLM)
- [meta-llama](#meta-llama)

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch

torch.cuda.get_device_name()

'Tesla P40'

In [2]:
import time
import random
import json
import numpy as np
from evaluate import load
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# Load dataset
dataset_name = "locchh/nvidia_qa"
dataset = load_dataset(dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 9082
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 1135
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1136
    })
})

## smolLM

In [4]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM-360M"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [5]:
# Function to calculate token lengths
def compute_token_lengths(dataset, field):
    return [len(tokenizer(answer, truncation=True, max_length=512)['input_ids']) for answer in dataset[field]]

# Analyze token lengths in the 'answer' column
train_lengths = compute_token_lengths(dataset["train"], "answer")
validation_lengths = compute_token_lengths(dataset["validation"], "answer")
test_lengths = compute_token_lengths(dataset["test"], "answer")

# Print statistics
print("Train Answer Lengths - Mean:", np.mean(train_lengths), "Max:", np.max(train_lengths))
print("Validation Answer Lengths - Mean:", np.mean(validation_lengths), "Max:", np.max(validation_lengths))
print("Test Answer Lengths - Mean:", np.mean(test_lengths), "Max:", np.max(test_lengths))

Train Answer Lengths - Mean: 32.277802246201276 Max: 109
Validation Answer Lengths - Mean: 32.461674008810576 Max: 84
Test Answer Lengths - Mean: 32.403169014084504 Max: 90


In [6]:
def generate_answer(question):
    # Tokenize the input and move tensors to GPU
    inputs = tokenizer(f"Question: {question} Answer in one sentence:", return_tensors="pt").to(device)
    # Generate output with a max token limit to encourage brevity
    outputs = model.generate(**inputs, max_new_tokens=50, no_repeat_ngram_size=2)
    # Decode the output and return the generated answer (after removing "Answer in one sentence:")
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer in one sentence:")[-1].strip()

In [7]:
answer = generate_answer(dataset["test"][0]["question"])
answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'Nvidia-GPT is a powerful language model that can be used to generate text, code, and other forms of content. It is particularly useful for tasks such as natural language processing, machine translation, chatbots, etc.\nQ:'

In [8]:
references = [item["answer"] for item in dataset["test"]]
predictions = []

# Populate the dataset with generated answers
for i,item in enumerate(dataset["test"]):
    print(f"{i+1}/{dataset['test'].num_rows}")
    predictions.append(generate_answer(item["question"]))
    time.sleep(1) # Cooling time

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


2/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

1134/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1135/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1136/1136


In [9]:
# Export json file
res = {
    "references": references,
    "predictions": predictions
}

# Write to JSON file
with open("test_smollm_360m.json", "w") as f:  # Use "w" mode to write to the file
    json.dump(res, f, indent=4)  # json.dump writes the data directly to the file

In [10]:
# Load evaluation metrics
bertscore = load("bertscore")
rouge = load("rouge")

# Read from JSON file
with open("test_smollm_360m.json", "r") as f:  # Use "r" mode to read the file
    loaded_res = json.load(f)

# Access data
references = loaded_res["references"]
predictions = loaded_res["predictions"]

In [11]:
# Evaluate with ROUGE
rouge_results = rouge.compute(predictions=predictions, references=references)

print("\nROUGE Results:")
print(f"ROUGE-1: {rouge_results['rouge1']}")
print(f"ROUGE-2: {rouge_results['rouge2']}")
print(f"ROUGE-L: {rouge_results['rougeL']}")


ROUGE Results:
ROUGE-1: 0.20126534818559824
ROUGE-2: 0.03509274114404221
ROUGE-L: 0.15079218771551753


In [ ]:
# Compute BERTScore
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")

print(f"Average BERTScore Precision: {np.mean(bertscore_results['precision'])}")
print(f"Average BERTScore Recall: {np.mean(bertscore_results['recall'])}")
print(f"Average BERTScore F1 Score: {np.mean(bertscore_results['f1'])}")

/home/loc/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/loc/miniconda3/envs/py38/lib/python3.8/site-packages/bert_score/utils.py:624: UserWarning: Truncating the start/stop/step of slice. This is likely because of saved old models when the start/stop/step were larger.
  emb = embs[i, :sequence_len]
/home/loc/miniconda3/envs/py38/lib/python3.8/sit

## meta-llama

In [4]:
# Load the model and tokenizer
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [5]:
# Function to calculate token lengths
def compute_token_lengths(dataset, field):
    return [len(tokenizer(answer, truncation=True, max_length=512)['input_ids']) for answer in dataset[field]]

# Analyze token lengths in the 'answer' column
train_lengths = compute_token_lengths(dataset["train"], "answer")
validation_lengths = compute_token_lengths(dataset["validation"], "answer")
test_lengths = compute_token_lengths(dataset["test"], "answer")

# Print statistics
print("Train Answer Lengths - Mean:", np.mean(train_lengths), "Max:", np.max(train_lengths))
print("Validation Answer Lengths - Mean:", np.mean(validation_lengths), "Max:", np.max(validation_lengths))
print("Test Answer Lengths - Mean:", np.mean(test_lengths), "Max:", np.max(test_lengths))

Train Answer Lengths - Mean: 31.64567275930412 Max: 103
Validation Answer Lengths - Mean: 31.787665198237885 Max: 77
Test Answer Lengths - Mean: 31.734154929577464 Max: 83


In [6]:
# Generate answers using the model

# def generate_answer(question):
#     # Tokenize the input and move tensors to GPU
#     inputs = tokenizer(f"Question: {question} Answer:", return_tensors="pt").to(device)
#     # Generate output and move it back to CPU for decoding
#     outputs = model.generate(**inputs, max_new_tokens=50)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[-1].strip()

def generate_answer(question):
    # Tokenize the input and move tensors to GPU
    inputs = tokenizer(f"Question: {question} Answer in one sentence:", return_tensors="pt").to(device)
    # Generate output with a max token limit to encourage brevity
    outputs = model.generate(**inputs, max_new_tokens=50, no_repeat_ngram_size=2)
    # Decode the output and return the generated answer (after removing "Answer in one sentence:")
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer in one sentence:")[-1].strip()

In [7]:
answer = generate_answer(dataset["test"][0]["question"])
answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


"NVIDIA's GPT is a cloud-based Gpt language model that enables businesses to leverage advanced AI capabilities, such as natural language processing and computer vision, to enhance their digital transformation.\n\nAnswer: The NVIDIA GTP is an enterprise-focused AI platform that provides"

In [8]:
dataset["test"].num_rows

1136

In [ ]:
references = [item["answer"] for item in dataset["test"]]
predictions = []

# Populate the dataset with generated answers
for i,item in enumerate(dataset["test"]):
    print(f"{i+1}/{dataset['test'].num_rows}")
    predictions.append(generate_answer(item["question"]))
    time.sleep(1) # Cooling time

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


2/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

98/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


99/1136


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


100/1136


In [ ]:
# Export json file
res = {
    "references": references,
    "predictions": predictions
}

# Write to JSON file
with open("test_llama_1b.json", "w") as f:  # Use "w" mode to write to the file
    json.dump(res, f, indent=4)  # json.dump writes the data directly to the file

In [ ]:
# Load evaluation metrics
bertscore = load("bertscore")
rouge = load("rouge")

# Read from JSON file
with open("test_llama_1b.json", "r") as f:  # Use "r" mode to read the file
    loaded_res = json.load(f)

# Access data
references = loaded_res["references"]
predictions = loaded_res["predictions"]

In [ ]:
# Evaluate with ROUGE
rouge_results = rouge.compute(predictions=predictions, references=references)

print("\nROUGE Results:")
print(f"ROUGE-1: {rouge_results['rouge1']}")
print(f"ROUGE-2: {rouge_results['rouge2']}")
print(f"ROUGE-L: {rouge_results['rougeL']}")

In [ ]:
# Compute BERTScore
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")

print(f"Average BERTScore Precision: {np.mean(bertscore_results['precision'])}")
print(f"Average BERTScore Recall: {np.mean(bertscore_results['recall'])}")
print(f"Average BERTScore F1 Score: {np.mean(bertscore_results['f1'])}")

/home/loc/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/loc/miniconda3/envs/py38/lib/python3.8/site-packages/bert_score/utils.py:624: UserWarning: Truncating the start/stop/step of slice. This is likely because of saved old models when the start/stop/step were larger.
  emb = embs[i, :sequence_len]
/home/loc/miniconda3/envs/py38/lib/python3.8/sit